# Dummy Net

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

from utils.data import init_model_logging
import tensorflow as tf
import numpy as np
import os

## Build Net Graph

In [ ]:
graph = tf.Graph()
with graph.as_default():
    with tf.name_scope('inputs'):
        x = tf.placeholder(tf.float32, shape=[None, 3], name='x')
        label = tf.placeholder(tf.float32, shape=[None, 1], name='label')
    
    with tf.variable_scope('layer_1'):
        w1 = tf.get_variable('w1', initializer=tf.truncated_normal(shape=[3,2], stddev=0.1))
        tf.summary.histogram('w1_histogram', w1)
        h = tf.matmul(x, w1)
    
    with tf.variable_scope('layer_2'):
        w2 = tf.get_variable('w2', initializer=tf.constant_initializer(0), shape=[2,1])
        tf.summary.histogram('w2_histogram', w2)
        y = tf.matmul(h, w2)
    
    with tf.name_scope('loss'):
        loss = tf.losses.mean_squared_error(label, y)
        tf.summary.scalar('loss_summary', loss)

    with tf.name_scope('training'):
        train_step = tf.train.GradientDescentOptimizer(learning_rate=0.005).minimize(loss)

    initialize_vars = tf.global_variables_initializer()
    merge_summaries = tf.summary.merge_all()

In [ ]:
graph.get_all_collection_keys()

## Init Model Logging

In [ ]:
base_dir = '/tensorboard_summaries/dummy_net/'
logging_meta = init_model_logging(base_dir, 'experiment1', graph=graph, remove_existing=True)

In [ ]:
logging_meta

## Debug Net

In [ ]:
session = tf.Session(graph=graph)
session.run(initialize_vars)

In [ ]:
feed_dict = {x:  np.array([[1,2,3]]), label: np.array([[5]])}
_w1, _h = session.run([w1, h], feed_dict)
print('w1:', _w1)

In [ ]:
session.close()

## Run net

In [ ]:
config = tf.ConfigProto(allow_soft_placement=True)
config.gpu_options.allow_growth = True

with tf.Session(graph=graph, config=config) as session:
    session.run(initialize_vars)
    
    for iteration in range(100):
        feed_dict = {x:  np.array([[1,2,3]]), label: np.array([[5]])}
        fetches = [train_step, y, loss, merge_summaries]
        
        _, _y, _loss, _summary = session.run(fetches, feed_dict)
        
        logging_meta['train_writer'].add_summary(_summary, iteration)
        print("Iteration {}: loss {}, y {}".format(iteration, _loss, _y[0][0]))

    logging_meta['saver'].save(session, logging_meta['model_path'])